# OneBoxMap helper

**Results limits:**
```
OneBoxMap.default_terms_limit = 3
OneBoxMap.default_results_limit = 10
OneBoxMap.default_suggestions_limit = 5
```

In [ ]:
from search.user import permissive
from search.widgets import OneBoxMap

OneBoxMap(permissive).run()

## User profile

Supports: 

- mandatory opt-in/out decision for position use and activity storing
- default position is that of IP if position use is opted-in
- default position is Berlin otherwise
- per country languages
- default language is that of the IP position locale if position use is opted-in

`permissive` default user profile has both `use_my_position` and `store_my_activity` set to `True`

## Application designs

`Design.one` and `SearchResultJson`

In [1]:
from search.user import UserProfile
from search.widgets import OneBoxMap, Design, SearchResultJson

user_profile = UserProfile(name="Emmanuel",
                           languages = {"DEU": "de", "AUT": "de", "FRA": "fr", "all": "en"},
                           use_my_position=True, 
                           store_my_activity=True)

app = OneBoxMap(
    user_profile=user_profile,
    design=Design.one, 
    resultlist_class=SearchResultJson)

app.run()

HTML(value='<style>.term-button button { font-size: 10px; }</style>')

## OneBoxMap specialization

`OneBoxCatNearCat` class is a client-side tweak:
- captures queries following the format _head nearby_conjunction tail_
- supports various languages and multiple nearby conjunction by language ("near", "close to", ...)
- uses Autosuggest service to check if _head_ and _tail_ are both about place categories
- Adds a category near category query suggestion button at the top of the Autosuggest suggestion list
- Sends such category near category query to HERE Location Graph instead of Discover service

In [ ]:
from search.user import permissive
from search.widgets import OneBoxCatNearCat

OneBoxCatNearCat(permissive).run()